# Lab: Spark Machine Learning

In this lab, you will build a simple ML pipeline and model with Spark Machine Learning.

Start your SparkSession:

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("lab-ml").getOrCreate()

In [2]:
spark

<a id="load"></a>
## 1. Load and explore data

Load the file `s3://bigdatateaching/misc/gosales_tx_naivebayes.csv` into a DataFrame:

In [3]:
df_data = spark.read\
  .format('csv')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load('s3://bigdatateaching/misc/gosales_tx_naivebayes.csv')

Print the schema of the DataFrame

In [4]:
df_data.printSchema()

root
 |-- PRODUCT_LINE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- PROFESSION: string (nullable = true)



Print the first few records:

In [5]:
df_data.show()

+--------------------+------+---+--------------+------------+
|        PRODUCT_LINE|GENDER|AGE|MARITAL_STATUS|  PROFESSION|
+--------------------+------+---+--------------+------------+
|Personal Accessories|     M| 27|        Single|Professional|
|Personal Accessories|     F| 39|       Married|       Other|
|Mountaineering Eq...|     F| 39|       Married|       Other|
|Personal Accessories|     F| 56|   Unspecified| Hospitality|
|      Golf Equipment|     M| 45|       Married|     Retired|
|      Golf Equipment|     M| 45|       Married|     Retired|
|   Camping Equipment|     F| 39|       Married|       Other|
|   Camping Equipment|     F| 49|       Married|       Other|
|  Outdoor Protection|     F| 49|       Married|       Other|
|      Golf Equipment|     M| 47|       Married|     Retired|
|      Golf Equipment|     M| 47|       Married|     Retired|
|Mountaineering Eq...|     M| 21|        Single|      Retail|
|Personal Accessories|     F| 66|       Married|       Other|
|   Camp

Count the number of records:

In [6]:
print("Number of records: " + str(df_data.count()))

Number of records: 60252


### 2.1: Prepare data

In this subsection you will split your data into: train, test and predict datasets. Create three splits of  `df_data` (train, test, predict) by using the `randomSplit` method:

In [7]:
splitted_data = df_data.randomSplit([0.8, 0.18, 0.02], 24)
train_data = splitted_data[0]
test_data = splitted_data[1]
predict_data = splitted_data[2]

print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))
print("Number of prediction records : " + str(predict_data.count()))

Number of training records: 48176
Number of testing records : 10860
Number of prediction records : 1216


### 2.2: Create pipeline and train a model

For this lab, your job is to build a model that classifies the `PRODUCT_LINE`. In this section you will create a machine learning pipeline and then train the model. The next cell imports all the packages you will need:

In [8]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In the following step, convert all the string fields to numeric ones. Look at the DataFrame structure to determine which ones you need to convert. Use the `StringIndexer` transformer. You need to create a transformer for each columnt you want to modify.

In [9]:
stringIndexer_label = StringIndexer(inputCol="PRODUCT_LINE", outputCol="label")#.fit(df_data)
stringIndexer_prof = StringIndexer(inputCol="PROFESSION", outputCol="PROFESSION_IX")
stringIndexer_gend = StringIndexer(inputCol="GENDER", outputCol="GENDER_IX")
stringIndexer_mar = StringIndexer(inputCol="MARITAL_STATUS", outputCol="MARITAL_STATUS_IX")

Try looking at the values for one of the re-encoded columns using the `labels` method. Does it work?

In [10]:
stringIndexer_label.labels

AttributeError: 'StringIndexer' object has no attribute 'labels'

To see the values, the transformer needs to be executed first, and you can do so by using the `fit` method. Try now:

In [11]:
si_label_fit = StringIndexer(inputCol="PRODUCT_LINE", outputCol="label").fit(df_data)

In [12]:
si_label_fit.labels

['Camping Equipment',
 'Personal Accessories',
 'Mountaineering Equipment',
 'Golf Equipment',
 'Outdoor Protection']

In the following step, create a feature vector by combining all string features together usinf the `vectorAssembler` method:

In [13]:
vectorAssembler_features = VectorAssembler(
    inputCols=["GENDER_IX", 
               "AGE", 
               "MARITAL_STATUS_IX", 
               "PROFESSION_IX"], 
    outputCol="features")

In [25]:
vectorAssembler_features

Param(parent='VectorAssembler_9cfff512691f', name='inputCols', doc='input column names.')

Next, define estimators you want to use for classification. You will build a Random Forest using the `RandomForestClassifier` estimator:

In [15]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

Finally, indexed labels back to original labels.

In [16]:
labelConverter = IndexToString(inputCol="prediction", 
                               outputCol="predictedLabel", 
                               labels=stringIndexer_label.fit(df_data).labels)

Let's build the pipeline now. A pipeline consists of transformers and an estimator.

In [17]:
pipeline_rf = Pipeline(stages=[stringIndexer_label, 
                               stringIndexer_prof, 
                               stringIndexer_gend, 
                               stringIndexer_mar, 
                               vectorAssembler_features, 
                               rf, labelConverter])

Now, you can train your Random Forest model by using the previously defined **pipeline** and **train data**.

In [18]:
train_data.printSchema()

root
 |-- PRODUCT_LINE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- PROFESSION: string (nullable = true)



In [19]:
train_data.show()

+-----------------+------+---+--------------+-----------+
|     PRODUCT_LINE|GENDER|AGE|MARITAL_STATUS| PROFESSION|
+-----------------+------+---+--------------+-----------+
|Camping Equipment|     F| 18|        Single|      Other|
|Camping Equipment|     F| 18|        Single|      Other|
|Camping Equipment|     F| 18|        Single|     Retail|
|Camping Equipment|     F| 18|        Single|     Retail|
|Camping Equipment|     F| 19|        Single|Hospitality|
|Camping Equipment|     F| 19|        Single|Hospitality|
|Camping Equipment|     F| 19|        Single|Hospitality|
|Camping Equipment|     F| 19|        Single|Hospitality|
|Camping Equipment|     F| 19|        Single|Hospitality|
|Camping Equipment|     F| 19|        Single|Hospitality|
|Camping Equipment|     F| 19|        Single|Hospitality|
|Camping Equipment|     F| 19|        Single|Hospitality|
|Camping Equipment|     F| 19|        Single|Hospitality|
|Camping Equipment|     F| 19|        Single|Hospitality|
|Camping Equip

In [20]:
model_rf = pipeline_rf.fit(train_data)

You can check your **model accuracy** now. To evaluate the model, use **test data**.

In [21]:
predictions = model_rf.transform(test_data)
evaluatorRF = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorRF.evaluate(predictions)

print("Accuracy = %g" % accuracy)
print("Test Error = %g" % (1.0 - accuracy))

Accuracy = 0.584807
Test Error = 0.415193


In [22]:
predictions.show()

+-----------------+------+---+--------------+-----------+-----+-------------+---------+-----------------+------------------+--------------------+--------------------+----------+--------------------+
|     PRODUCT_LINE|GENDER|AGE|MARITAL_STATUS| PROFESSION|label|PROFESSION_IX|GENDER_IX|MARITAL_STATUS_IX|          features|       rawPrediction|         probability|prediction|      predictedLabel|
+-----------------+------+---+--------------+-----------+-----+-------------+---------+-----------------+------------------+--------------------+--------------------+----------+--------------------+
|Camping Equipment|     F| 18|        Single|      Other|  0.0|          0.0|      1.0|              1.0|[1.0,18.0,1.0,0.0]|[4.33081888741225...|[0.21654094437061...|       1.0|Personal Accessories|
|Camping Equipment|     F| 18|        Single|     Retail|  0.0|          7.0|      1.0|              1.0|[1.0,18.0,1.0,7.0]|[3.67155294270736...|[0.18357764713536...|       1.0|Personal Accessories|
|Camp

Adapted from [this notebook from IBM Watson.](https://dataplatform.ibm.com/analytics/notebooks/89492fd6-a641-4819-9176-3d9381561df9/view?access_token=d80bef1a172d1d83d3721b101886337158457281774186f181a2e6a5b57f5ec7#)